In [1]:
from ibm_botocore.client import Config
import ibm_boto3
import pandas as pd
import re

In [2]:
credentials={
  "apikey": "f18U20Myrg3oGE_31dJH9B7wfXY3ug_wgLYELxs8NVTr",
  "endpoints": "https://control.cloud-object-storage.cloud.ibm.com/v2/endpoints",
  "iam_apikey_description": "Auto-generated for key a9c832e8-e31a-4109-b20f-a80955cb2537",
  "iam_apikey_name": "Project-crew",
  "iam_role_crn": "crn:v1:bluemix:public:iam::::serviceRole:Writer",
  "iam_serviceid_crn": "crn:v1:bluemix:public:iam-identity::a/60a7ab6659c9400f866dc0bf936cfd45::serviceid:ServiceId-adc9a8a6-d51d-4a6f-b4fc-f05d82c25eef",
  "resource_instance_id": "crn:v1:bluemix:public:cloud-object-storage:global:a/60a7ab6659c9400f866dc0bf936cfd45:2338ce79-3cbe-4e13-bd16-880ba267a455::"
}
storage = ibm_boto3.client(service_name='s3',
    ibm_api_key_id=credentials['apikey'],
    ibm_service_instance_id=credentials['resource_instance_id'],
    ibm_auth_endpoint='https://iam.cloud.ibm.com/oidc/token',
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.eu-gb.cloud-object-storage.appdomain.cloud')

In [3]:
storage.download_file(Bucket='master-kahdeksas',Key='customers_1548068768342.json',Filename='asiakkaat.json')

In [4]:
storage.download_file(Bucket='master-kahdeksas',Key='assignments_1548069355662.json',Filename='toimeksiannot.json')

In [17]:
!ls -l

total 28980
-rw-r----- 1 dsxuser dsxuser    49410 Jun  3 09:20 agentit.json
-rw-r----- 1 dsxuser dsxuser 22837390 Jun  3 10:30 asiakkaat.json
-rw-r----- 1 dsxuser dsxuser  6779679 Jun  3 10:30 toimeksiannot.json


## Customers

In [18]:
customers_df=pd.read_json('asiakkaat.json')

In [19]:
customers_df.drop(labels=['AD_ALLOWED_FLAG','AGENCY_ID','CONTACT_COUNTRY_CODE', 'CONTACT_EMAIL','CONTACT_FAX'],axis='columns',inplace=True)
customers_df.drop(labels=['CONTACT_FIRSTNAME', 'CONTACT_LASTNAME', 'CONTACT_MOBILE_PHONE'],axis='columns',inplace=True)
customers_df.drop(labels=['CONTACT_PERSON_ID', 'CONTACT_PHONE', 'CONTACT_POSTAREA'],axis='columns',inplace=True)
customers_df.drop(labels=['CONTACT_POSTCODE', 'CONTACT_SSN', 'CONTACT_STREET', 'CONTACT_TITLE'],axis='columns',inplace=True)
customers_df.drop(labels=[ 'FAX', 'INFO','MOBILE_PHONE', 'PHONE','DOOR_NUMBER', 'EMAIL','HIDDEN_FLAG'],axis='columns',inplace=True)
#customers_df=customers_df[ customers_df['SSN'].isna() == False ]# Drop customers, that don't have social security number
customers_df['AGENT_EMAIL'].replace('(^\s+)|(\s+$)','',regex=True,inplace=True)#Trim spaces
customers_df['COMPANY_NAME'].replace('(^\s+)|(\s+$)','',regex=True,inplace=True)#Trim spaces
customers_df['FIRSTNAME'].replace('(^\s+)|(\s+$)','',regex=True,inplace=True)#Trim spaces
customers_df['LASTNAME'].replace('(^\s+)|(\s+$)','',regex=True,inplace=True)#Trim spaces
customers_df['ESTATE_NAME'].replace('(^\s+)|(\s+$)','',regex=True,inplace=True)#Trim spaces
customers_df['POSTAREA'].replace('(^\s+)|(\s+$)','',regex=True,inplace=True)#Trim spaces
customers_df['STREET'].replace('(^\s+)|(\s+$)','',regex=True,inplace=True)#Trim spaces
customers_df['SSN'].replace('(^\s+)|(\s+$)','',regex=True,inplace=True)#Trim spaces
customers_df[['CREATEDATE','UPDATEDATE']]=customers_df[['CREATEDATE','UPDATEDATE']].apply(pd.to_datetime)#conversion to datetime-object
#customers_df

## Assignments

In [20]:
assignments_df=pd.read_json('toimeksiannot.json')

In [21]:
assignments_df.drop(labels=['AGENCY_ID'],axis='columns',inplace=True)#All IDs same -> drop the column

In [22]:
buyer_or_seller=[]

for row in assignments_df['BUYERS']:
    buyer_or_seller.extend(row)
for row in assignments_df['SELLERS']:
    buyer_or_seller.extend(row)
print("Customers in either role: {}".format(len(buyer_or_seller)))
buyer_or_seller=set(buyer_or_seller) # make unique
print("Unique customers: {}".format(len(buyer_or_seller)))



Customers in either role: 14373
Unique customers: 11696


## Drop old records
### Older than ten years

In [23]:
ten_years_ago=pd.to_datetime('today') - pd.DateOffset(years=10)
drop_10_year_df=customers_df[customers_df['CREATEDATE']<ten_years_ago]

In [24]:
drop_10_year_df.shape

(414, 23)

### One year without assignment

In [25]:
customers_df=customers_df[customers_df['CREATEDATE']>ten_years_ago]
one_year_ago=pd.to_datetime('today') - pd.DateOffset(years=1)
does_not_have_assignment=[]
for ind in customers_df.index: 
    does_not_have_assignment.append(not customers_df.loc[ind,'CUSTOMER_ID'] in buyer_or_seller)
not_assignment_df=customers_df[ does_not_have_assignment]
drop_1_year_df=not_assignment_df[not_assignment_df['CREATEDATE']<one_year_ago]

In [26]:
drop_1_year_df.shape

(8581, 23)